In [1]:
# Must do it this way to use keras-vis https://github.com/keisen/tf-keras-vis/blob/master/examples/attentions.ipynb
!pip install --upgrade tf-keras-vis tensorflow matplotlib

# Import
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, MaxPool2D
from tensorflow.keras.models import Model


from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils import normalize


Requirement already up-to-date: tf-keras-vis in /usr/local/lib/python3.7/dist-packages (0.5.5)
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.7/dist-packages (2.4.1)
Requirement already up-to-date: matplotlib in /usr/local/lib/python3.7/dist-packages (3.3.4)


In [2]:
# Load data
data = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = data.load_data()

print("Train X data shape: ", x_train.shape)
print("Train Y data shape: ", y_train.shape)

print("Test data shape: ", x_test.shape)
print("Test Y data shape: ", y_test.shape)

# Normalize data
x_train, x_test = x_train / 255.00, x_test / 255.00 

# Add color chanel to x_train and x_test datashape 
x_train = np.expand_dims(x_train, -1)
print("Reshaped Train X data shape: ", x_train.shape)

x_test = np.expand_dims(x_test, -1)
print("Reshaped Test data shape: ", x_test.shape)


Train X data shape:  (60000, 28, 28)
Train Y data shape:  (60000,)
Test data shape:  (10000, 28, 28)
Test Y data shape:  (10000,)
Reshaped Train X data shape:  (60000, 28, 28, 1)
Reshaped Test data shape:  (10000, 28, 28, 1)


In [3]:
# Define a function to predict saliency map
def predict_saliency(data_shape):
  input = Input(shape=(data_shape))
  conv_1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(input)
  max_pool1 = MaxPool2D(pool_size=(2, 2))(conv_1)
  conv_2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(max_pool1)
  max_pool2 = MaxPool2D(pool_size=(2, 2))(conv_2)
  conv_3 = Conv2D(128, kernel_size=(3, 3), activation='relu')(max_pool2)
  flatten = Flatten()(conv_3)
  drop_out1 = Dropout(0.2)(flatten)
  dense1 = Dense(512, activation='relu')(drop_out1)
  drop_out2 = Dropout(0.2)(dense1)
  dense2 = Dense(10, activation='softmax', name='output_layer')(drop_out2)

  # Create the model
  model = Model(input, dense2)
  return model


In [4]:
# Define the Hyperparameters and Configs
input_data_shape = x_train[0].shape
# number_of_classes = y_test
number_of_epochs = 30


# Get the model and summary
model = predict_saliency(input_data_shape)
model.summary()

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.sparse_categorical_crossentropy, # Using sparse categorical cross entropy loss, so we do not need to convert y_train/y_test to_categorical 
              metrics=['accuracy'])



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0     

In [5]:
# Fit and train model
res = model.fit(x_train, y_train, validation_data=(x_test, y_test),
          epochs=number_of_epochs)

Epoch 1/30
1875/1875 [==============================] - 17s 4ms/step - loss: 0.6381 - accuracy: 0.7663 - val_loss: 0.3517 - val_accuracy: 0.8724
Epoch 2/30
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3146 - accuracy: 0.8836 - val_loss: 0.3005 - val_accuracy: 0.8912
Epoch 3/30
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2728 - accuracy: 0.8977 - val_loss: 0.2963 - val_accuracy: 0.8884
Epoch 4/30
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2469 - accuracy: 0.9076 - val_loss: 0.2587 - val_accuracy: 0.9066
Epoch 5/30
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2210 - accuracy: 0.9171 - val_loss: 0.2640 - val_accuracy: 0.9064
Epoch 6/30
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2019 - accuracy: 0.9233 - val_loss: 0.2599 - val_accuracy: 0.9059
Epoch 7/30
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1813 - accuracy: 0.9321 - val_loss: 0.2594 - val_accuracy

In [ ]:
def model_modifier(m):
    m.layers[-1].activation = tf.keras.activations.linear
    return m

def loss(output):
    # 1 is the imagenet index corresponding to Goldfish, 294 to Bear and 413 to Assault Rifle.
    return (output[1], output[294], output[413])

indices_to_visualize = [ 0, 12, 38, 83, 112, 74, 190]

saliency = Saliency(model,
                    model_modifier=model_modifier,
                    clone=False)
saliency_map = saliency(loss, x_train[1])